# Problem 2
In this section, we will do some feature engineering.  
First, preprocess the data in the same way of task1.
In this section, we try to avoid be overfitting to the training set. So, first we drop columns have less than 80% data. After that, we find that some columns have the same values such as 'Air Aspir Method' and 'Air Aspiration Method Desc'.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

def xlsx_to_csv_pd(path):
    data_train = pd.read_excel(path + '15.xlsx')
    xls_16 = pd.read_excel(path + '16.xlsx')
    xls_17 = pd.read_excel(path + '17.xlsx')
    data_train = pd.concat([data_train,xls_16,xls_17])
    data_test = pd.read_excel(path + '18.xlsx')
    return data_train, data_test

def x_y_split(df):
    y = df.pop('Comb Unrd Adj FE - Conventional Fuel')
    return df,y

def choose_column(df):
    count = df.shape[0] - df.count()
    filtered =  count[count < len(df)*0.2]
    index = list(filtered.index)
    index = [col for col in index if col.find('EPA') == -1 and col.find('FE') == -1 and col.find('MPG') == -1 and col.find('CO2') == -1 and col.find('Smog') == -1 and col.find('Guzzler') == -1 and col.find('Release Date') == -1 and col.find('Mfr Name') == -1 and col.find('Verify Mfr Cd') == -1]
    index = [col for col in index if col.find('Desc') == -1 or col.find('Calc Approach Desc') > -1 or col.find('Var Valve Timing Desc') > -1]
    return df[index], index

#load data
train, test = xlsx_to_csv_pd('/Users/haikundu/Desktop/4995AML/hw3/')

#data target split
train_x, train_y = x_y_split(train)
test_x, test_y = x_y_split(test)

#choose column
filtered_train, column_name = choose_column(train_x)
filtered_test = test[column_name]

#fill missing value and split data into categorical and numerical data
def fillna_mean(dfo,index,dfto):
    df = pd.DataFrame()
    dft = pd.DataFrame()
    cate = []
    num = []
    for col in index:
        if dfo[col][0].dtype == 'int64' or dfo[col][0].dtype == 'float64':
            df[col] = dfo[col].fillna(dfo.mean()[col])
            dft[col] = dfto[col].fillna(dfo.mean()[col])
            num.append(col)
        else:
            df[col] = dfo[col].fillna(dfo.mode()[col].tolist()[0]).tolist()
            dft[col] = dfto[col].fillna(dfo.mode()[col].tolist()[0]).tolist()
            cate.append(col)
    return df,dft,cate,num

full_train,full_test,cate,num = fillna_mean(filtered_train,column_name,filtered_test)
cate_train = full_train[cate]
cate_test = full_test[cate]
num_train = full_train[num]
num_test = full_test[num]

After preprocessing, we split the whole dataset into categorical set and numerical set.  
In this section, we adopt two ways for feature engineering.  
First, we applied one-hot-encoding method for categorical data. And create polynomial feature from numerical data.

In [2]:
def one_hot_encoding(train,test):
    full = pd.concat([train, test]) 
    extended = pd.get_dummies(full)
    boundry = len(train)
    train = extended[:boundry]
    test = extended[boundry:]
    return train, test

#one hot encoding for categorical data
train_cat,test_cat = one_hot_encoding(cate_train,cate_test)
train_cat.index=list(range(0,len(train)))

#Build non-linear features for numerical variables
poly = PolynomialFeatures(2,include_bias= False)
train_num = pd.DataFrame(poly.fit_transform(num_train))
test_num = pd.DataFrame(poly.fit_transform(num_test))

#combine two sub-dataset
ohe_train = pd.concat([train_cat,train_num],axis=1)
ohe_test = pd.concat([test_cat,test_num],axis=1)

#Standard Scaler
scaler = StandardScaler()
scaler.fit(ohe_train)
X_train_scaled = scaler.transform(ohe_train)
X_test_scaled = scaler.transform(ohe_test)

In [3]:
# grid search for best parameter and exam the performance of ridge regression model on test set
param_grid_ridge = {'alpha': np.logspace(-3, 3, 20)}
grid_ridge = GridSearchCV(Ridge(),param_grid=param_grid_ridge,cv=10,
                          return_train_score=True)
grid_ridge.fit(X_train_scaled, train_y)
print("best mean cross-validation score: {:.3f}".format(grid_ridge.best_score_))
print("best parameters: {}".format(grid_ridge.best_params_))
print("test-set score: {:.3f}".format(grid_ridge.score(X_test_scaled, test_y)))

best mean cross-validation score: 0.976
best parameters: {'alpha': 2.976351441631316}
test-set score: 0.858


We can find that the validation score for ridge regression model improved. However, the test score decreased. Based on this result, we assume that the this model may be overfitting to the training set. At the same time, we found that there are too many variables comparing to the number of training data.   
So, we designed a second method for feature engineering.  
In this method, first we applied count based encoding method on categorical data. Then create polynomial features from the combined whole dataset.

In [4]:
#count based encoding
def cate_to_num(train,test,index):
    '''
    param:train. type: dataframe
    param:test.  type: dataframe
    param:col.   type: list of string
    '''
    boundry = len(train)
    full = pd.concat([train, test])
    result = pd.DataFrame()
    for col in index:
        result[col] = full.groupby(col)[col].transform('count')/len(full)
    train = result[:boundry]
    test = result[boundry:]
    return train, test

#count based encoding for categorical data
train_count,test_count = cate_to_num(cate_train,cate_test,cate)

#combine two sub-dataset
all_train = pd.concat([train_count,num_train],axis=1)
all_test = pd.concat([test_count,num_test],axis=1)

#Build non-linear features for numerical variables
poly_cb = PolynomialFeatures(2,include_bias= False)
train_all = pd.DataFrame(poly_cb.fit_transform(all_train))
test_all = pd.DataFrame(poly_cb.fit_transform(all_test))

#Standard Scaler
scaler = StandardScaler()
scaler.fit(train_all)
X_train_scaled_cb = scaler.transform(train_all)
X_test_scaled_cb = scaler.transform(test_all)

In [7]:
# grid search for best parameter and exam the performance of ridge regression model on test set
grid_ridge_cb = GridSearchCV(Ridge(),param_grid=param_grid_ridge,cv=10,
                          return_train_score=True)
grid_ridge_cb.fit(X_train_scaled_cb, train_y)
print("best mean cross-validation score: {:.3f}".format(grid_ridge_cb.best_score_))
print("best parameters: {}".format(grid_ridge_cb.best_params_))
print("test-set score: {:.3f}".format(grid_ridge_cb.score(X_test_scaled_cb, test_y)))

best mean cross-validation score: 0.977
best parameters: {'alpha': 2.976351441631316}
test-set score: 0.717


From the result, we find that both ways of feature engineering are overfitting to the training set. However, for the second method, there is an increasing for the test score comparing to score in the same way of encoding categorical data without feature engineering. So, the decreasing of test socre maybe due to count-based encoding method.